## Get dataset

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

In [2]:
from google.colab import drive

# Montar Google Drive en /content/drive
drive.mount('/content/drive')

# Listar archivos en el directorio raíz de Google Drive
!ls '/content/drive/MyDrive/dataset'

Mounted at /content/drive
face_class_model.pth  test  train  val


In [3]:
path = '/content/drive/MyDrive/dataset'
# path = '../data/final'

## Settings

In [4]:
seed = 21
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [5]:
# Settings
data_dir = path

batch_size = 32
num_epochs = 30
learning_rate = 0.001

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"[+] Device: {device}")

[+] Device: cuda


## Pre - processing

In [6]:
data_transforms = {
    'train': transforms.Compose([
        # transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        # transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        # transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [7]:
# Load the datasets with ImageFolder
image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), transform=data_transforms[x])
    for x in ['train', 'val', 'test']
}


In [8]:
# DataLoaders
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True if x == 'train' else False, num_workers=4)
    for x in ['train', 'val', 'test']
}

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## Architecture

In [9]:
# Class names
class_names = image_datasets['train'].classes
num_classes = len(class_names)

num_classes

25

In [10]:
# Load a pre-trained model
model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)

# Modify the last layer
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 105MB/s]


In [11]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [12]:
# Function to train the model and return the best model
def train_model(model, dataloaders, criterion, optimizer, num_epochs: int):
    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data
            for inputs, labels in tqdm(dataloaders[phase], desc=f"{phase} Phase"):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # Reset gradients
                optimizer.zero_grad()

                # Forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # Backward and optimize
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # Statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f"{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

            # Save the best model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = model.state_dict()

    print(f"Best val Acc: {best_acc:.4f}")

    # Load the best model weights
    model.load_state_dict(best_model_wts)
    return model

In [13]:
def test_model(model, dataloader):
    model.eval()
    running_corrects = 0

    with torch.no_grad():
        for inputs, labels in tqdm(dataloader, desc="Test Phase"):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            running_corrects += torch.sum(preds == labels.data)

    test_acc = running_corrects.double() / len(dataloader.dataset)
    print(f"Test Accuracy: {test_acc:.4f}")

In [14]:
model = train_model(model, dataloaders, criterion, optimizer, num_epochs)


Epoch 1/30


train Phase:   0%|          | 0/114 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
train Phase: 100%|██████████| 114/114 [05:25<00:00,  2.85s/it]


train Loss: 1.0036 Acc: 0.7139


val Phase: 100%|██████████| 15/15 [00:40<00:00,  2.71s/it]


val Loss: 1.3026 Acc: 0.6982

Epoch 2/30


...

val Loss: 0.2384 Acc: 0.9361

Epoch 30/30


train Phase: 100%|██████████| 114/114 [00:39<00:00,  2.87it/s]


train Loss: 0.0178 Acc: 0.9950


val Phase: 100%|██████████| 15/15 [00:02<00:00,  6.76it/s]

val Loss: 0.2623 Acc: 0.9383
Best val Acc: 0.9537


In [15]:
# Save model
torch.save(model.state_dict(), f'{path}/face_class_model2.pth')

In [16]:
test_model(model, dataloaders['test'])

Test Phase: 100%|██████████| 15/15 [00:44<00:00,  2.95s/it]

Test Accuracy: 0.9530


## Testing model

In [17]:
import torch
from torchvision import models, transforms
from PIL import Image

In [18]:
import os

image_path = os.path.join(os.getcwd(), 'dataset', 'test', 'armando_garcia', 'armando_garcia_3.jpg')
model_path = os.path.join(os.getcwd(), 'models', 'face_class_model.pth')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [19]:
# Transformaciones (iguales a las usadas en 'test')
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Redimensionar la imagen
    transforms.ToTensor(),         # Convertir la imagen a tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalizar
])

In [20]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")  # Transform to RGB
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension
    return input_tensor


def load_model(path, num_classes):
    model = models.resnet50(pretrained=False)
    model.fc = torch.nn.Linear(model.fc.in_features, num_classes)
    model.load_state_dict(torch.load(path, map_location=device))    # Load weights
    model = model.to(device)                                        # Send to GPU or CPU
    model.eval()
    return model


def predict_image(model, image_tensor, class_names):
    image_tensor = image_tensor.to(device)
    with torch.no_grad():
        output = model(image_tensor)
        _, predicted_class = torch.max(output, 1)
    return class_names[predicted_class.item()]

In [22]:
# class_names = os.listdir(os.path.join(os.getcwd(), 'dataset', 'test'))
class_names = '/content/drive/MyDrive/dataset/test'

In [23]:
# Load the image and preprocess it
input_tensor = preprocess_image(image_path)

# Load the model
model = load_model(model_path, num_classes=len(class_names))

# Get the prediction
prediction = predict_image(model, input_tensor, class_names)

# Show the prediction
print(f"La predicción para la imagen es: {prediction}")

FileNotFoundError: [Errno 2] No such file or directory: '/content/dataset/test/armando_garcia/armando_garcia_3.jpg'

## Assessment

In [ ]:
def calculate_accuracy(model, dataloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    print(f"Accuracy: {accuracy:.4f}")

# Usar con el conjunto de prueba:
calculate_accuracy(model, dataloaders['test'])


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

def plot_confusion_matrix(model, dataloader, class_names):
    all_labels = []
    all_preds = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
    disp.plot(cmap='Blues')
    return cm

# Llamar a la función
plot_confusion_matrix(model, dataloaders['test'], class_names)


In [ ]:
from sklearn.metrics import classification_report

def evaluate_classification_report(model, dataloader, class_names):
    all_labels = []
    all_preds = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    report = classification_report(all_labels, all_preds, target_names=class_names)
    print(report)

# Llamar a la función
evaluate_classification_report(model, dataloaders['test'], class_names)
